# IronHacks Submission Notebook

## Background
---

In this project, the aim is to make a prediction of foot traffic for week 44 (Since Indiana recorded first COVID-19 case) in 1804 Point of Interests (POIs) in Tippecanoe County in Indiana, United States of America (a regression task). The data that is being used has been collected from the week 1 to week 43 and has is available as a BIgQuery project file. The schema of the tables that will be used for this project can be found [here](https://docs.google.com/spreadsheets/d/e/2PACX-1vQUgT-CyXoPO6Fa4r4YlwF6uqHMvcqiWM4UNlUqYNdrljntzZsL8sU9-BIZZIOdd_CO3W5ILB_MW4TW/pubhtml).

To start with, the six tables would be imported from the BigQuery project and stored as a pandas DataFrame. Then, Exploratory Data Analysis (EDA) would be carried out on the data to better understand their relationships for feature selection for the modeling task.

In [4]:
%%capture
%logstop
%logstart -t -r -q ipython_command_log.py global

#- IRONHACKS RESEARCH TRACKING CODE
#----------------------------------
# The following code is used to help our research team understand how you 
# our notebook environment. We do not collect any personal information with
# the following code, it is used to measure when and how often you work on
# your submission files.

import os
from datetime import datetime
import IPython.core.history as history

ha = history.HistoryAccessor()
ha_tail = ha.get_tail(1)
ha_cmd = next(ha_tail)
session_id = str(ha_cmd[0])
command_id = str(ha_cmd[1])
timestamp = datetime.utcnow().isoformat()
history_line = ','.join([session_id, command_id, timestamp]) + '\n'
logfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')
logfile.write(history_line)
logfile.close()

### Imports

In [5]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import os
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='keys.json'   #using unique key to get access to the data 
bigquery_client = bigquery.Client(project='ironhacks-covid19-data')
bigquery_client
bigquery_client = bigquery.Client()

def table_pandas(table):  #function to convert the six tables to pandas DataFrame
    QUERY = """

    SELECT *
    FROM ironhacks-covid19-data.ironhacks_covid19_competition.{i}

    """.format(i=table)

    query_job = bigquery_client.query(QUERY)
    return query_job.to_dataframe()

In [ ]:
main_data = table_pandas('weekly_patterns')
mobility = table_pandas('mobility_graph')
social_distancing = table_pandas('cbg_social_distancing')
covid_cases = table_pandas('covid19_cases')
executive_orders = table_pandas('executive_orders')
predict_table = table_pandas('prediction_list_poi')

In [7]:
main_data = table_pandas('weekly_patterns')
predict_table = table_pandas('prediction_list_poi')

In [8]:
covid_cases = table_pandas('covid19_cases')

In [13]:
mobility = table_pandas('mobility_graph')

### Exploratory Data Analysis

With an aim to get further understand the available data and know which of features to use in modeling, each table will be explored in the next couple of cells.  

In [7]:
def table_head (table):   #function for getting familiar with the table
    return table.head()

In [9]:
table_head(covid_cases)

,week_number,start_date,county,fips,cases,deaths
0,11,2020-03-16,Tippecanoe,18157,7,0
1,12,2020-03-23,Tippecanoe,18157,46,3
2,13,2020-03-30,Tippecanoe,18157,148,7
3,14,2020-04-06,Tippecanoe,18157,278,7
4,15,2020-04-13,Tippecanoe,18157,352,12


In [17]:
len(mobility['poi_id'].unique())

1803

In [ ]:
table_head(predict_table)

### Working with The Primary Data
For a start, the main_data will be pre-processed for the modeling and used.

In [9]:
x = main_data.copy()

x['date_start'] =  pd.to_datetime(x['date_start'], format='%Y-%m-%d') #convert to datetime

In [10]:
x['dayofweek'] = pd.DatetimeIndex(x['date_start']).dayofweek
x['is_weekend'] = x.dayofweek.isin([5,6])*1
x['quarter'] = x['date_start'].dt.quarter
x['month'] = x['date_start'].dt.month
x['day'] = x['date_start'].dt.day
x['weekofyear'] = x['date_start'].dt.weekofyear
x['yearday'] = x['date_start'].dt.dayofyear

In [11]:
unique_poi = x['poi_id'].unique()

In [12]:
%%capture
x.set_index('poi_id', inplace=True)
# features = x[['week_number', 'dayofweek', 'is_weekend', 'quarter', 'month', 'day', 'weekofyear', 'yearday', 'raw_visit_counts']]
target = x['raw_visit_counts']
x.drop(columns=['raw_visit_counts', 'date_start'], inplace=True)

In [13]:
#log transform of target
target = np.log1p(target)

In [ ]:
x.info()

#### Dealing with missing values (brands, top_category and NAICS)

In [14]:
x.drop('brands', axis=1, inplace=True)  #droping since a significant number is null

In [15]:
x = x.astype({'NAICS' : 'float64'})  #convert NAICS to float format

In [16]:
#filling the null values with the mode
x['NAICS'].fillna(x['NAICS'].mode()[0], inplace=True)

x['top_category'].fillna(x['top_category'].mode()[0], inplace=True)

In [ ]:
x.info()

#### Using CatBoost

In [17]:
cate_features_index=np.where(x.dtypes != float) [0] 
cate_features_index

array([ 0,  3,  4,  5,  7,  9, 12, 13, 14, 15, 16, 17, 18])

In [45]:
def cat_regressor(features, target):
    """
    Function to model the foot traffic time series data.
    It uses CatBoost Regressor model for forecasting.
    
    Returns
        forecasts (float):  Forecast value for the next time step in the series
        y (int): True value to be compared with the forecasted value
    """    
    
    y = target.shift(periods=-1)


#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    dtregr = CatBoostRegressor()
    dtregr.fit(features[:-2], y[:-2], cat_features=cate_features_index)
    cat_grid = {
        'iterations': np.arange(30, 50, 100),
        'depth': np.arange(6, 8, 10),
        'learning_rate': [0.01, 0.05, 0.1]
    }
    
    cat_model_rs = GridSearchCV(estimator= dtregr,
                                     param_grid=cat_grid,
                                     cv=2,
                                     scoring = ['neg_mean_squared_error','neg_mean_absolute_error'],
                                     refit = 'neg_mean_absolute_error',
                                     verbose=True)
    
    cat_model_rs.fit(features[:-2], y[:-2], cat_features=cate_features_index)

    forecasts = cat_model_rs.predict(np.array(features.iloc[-2]).reshape(1,-1))
    
    return forecasts, y.iloc[-2]

In [ ]:
%%capture

forecasts2 = []
tests = []
location_errors = []

for poi in unique_poi[1802:]:
    unique_X = x[x.index == poi]
    unique_y = target[target.index == poi]
#     df = weekly_patterns[weekly_patterns['poi_id'] == location]
#     df['exec_orders'] = df['week_number'].isin(order_weeks).astype(int)
#     try:
    f, t = cat_regressor(unique_X, unique_y)
    forecasts2.append(f)
    tests.append(t)
        
#         if idx % 200 == 0:
#             print("{} locations done.".format(idx))
        
#     except Exception as e:
#         print(e)
#         break;
        
forecasts2 = [forecast[0] for forecast in forecasts2]

In [62]:
forecasts2 = np.expm1(forecasts2)

In [82]:
forecasts2 = [int(np.round(i)) if i > 0 else 0 for i in forecasts2]

In [84]:
predict_table['poi_id'] = unique_poi
predict_table['raw_visit_counts'] = forecasts2
predict_table.to_csv('submission_prediction_output8.csv', index=False)